## 직업별 월급 평균 및 남여 종사자 수 표 만들기

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns

In [3]:
pip install pyreadstat

Note: you may need to restart the kernel to use updated packages.


In [4]:
# 데이터 불러오기
raw_welfare = pd.read_spss('Koweps_hpwc14_2019_beta2.sav')

In [5]:
# 복사본 만들기
welfare = raw_welfare.copy()

In [6]:
welfare.head()

,h14_id,h14_ind,h14_sn,h14_merkey,h_new,h14_cobf,p14_wsc,p14_wsl,p14_wgc,p14_wgl,...,wc14_64,wc14_65,wc14_5aq4,wc14_5aq5,wc14_5aq6,h14_pers_income1,h14_pers_income2,h14_pers_income3,h14_pers_income4,h14_pers_income5
0,2.0,1.0,1.0,20101.0,0.0,NaN,0.291589,0.291589,1307.764781,1307.764781,...,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
1,3.0,1.0,1.0,30101.0,0.0,NaN,0.419753,0.419753,1882.570960,1882.570960,...,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
2,4.0,1.0,1.0,40101.0,0.0,NaN,0.265263,0.265980,1189.691668,1192.908537,...,NaN,,NaN,NaN,NaN,NaN,1284.0,NaN,0.0,NaN
3,6.0,1.0,1.0,60101.0,0.0,NaN,0.494906,0.495941,2219.630833,2224.273816,...,1.0,.,2.0,4.0,4.0,2304.0,NaN,1800.0,0.0,NaN
4,6.0,1.0,1.0,60101.0,0.0,NaN,1.017935,1.017935,4565.389177,4565.389177,...,1.0,.,1.0,5.0,2.0,NaN,NaN,NaN,0.0,NaN


In [21]:
# 성별, 생년, 결혼 유무, 종교, 수입(연봉), 직업 분류, 지역
welfare = welfare.rename(
    columns = {'h14_g3' : 'sex',
              'h14_g4' : 'birth',
              'h14_g10' : 'marriage_type',
              'h14_g11' : 'religion',
              'p1402_8aq1' : 'income',
              'h14_eco9' : 'code_job',
              'h14_reg7' : 'code_region'
              })

In [8]:
# 성별 항목 이름 부여
welfare['sex'] = np.where(welfare['sex'] == 1, 'male', 'female')

In [12]:
# 직업명 불러오기
list_job = pd.read_excel('Koweps_Codebook_2019.xlsx', sheet_name='직종코드')

In [13]:
# welfare와 list_job 겹합(직업코드에 해당하는 직업명 넣기)
welfare = welfare.merge(list_job, how='left', on='code_job')

In [14]:
# welfare에 직업코드와 직업명 출력
welfare[['code_job', 'job']].head()

,code_job,job
0,NaN,NaN
1,NaN,NaN
2,762.0,전기공
3,855.0,금속기계 부품 조립원
4,NaN,NaN


In [15]:
# welfare에 직업코드에서 결측치 제거
welfare.dropna(subset=['code_job'])[['code_job', 'job']].head()

,code_job,job
2,762.0,전기공
3,855.0,금속기계 부품 조립원
7,941.0,청소원 및 환경미화원
8,999.0,기타 서비스 관련 단순 종사자
14,312.0,경영 관련 사무원


In [32]:
# 직업별 월급 평균표 만들기
job_income = welfare.dropna(subset=['job', 'income']) \
                    .groupby('job', as_index=False) \
                    .agg(mean_income = ('income', 'mean'))

In [33]:
job_income.head()

,job,mean_income
0,가사 및 육아 도우미,92.455882
1,간호사,265.219178
2,감정∙기술영업및중개관련종사자,391.000000
3,건물 관리원 및 검표원,168.375000
4,건설 및 광업 단순 종사자,261.975000


In [42]:
# 남성 직업 빈도 상위 10개 추출
job_male = welfare.dropna(subset=['job'])\
                .query('sex=="male"')\
                .groupby('job', as_index=False)\
                .agg(n=('job', 'count'))
                
job_male

,job,n
0,가사 및 육아 도우미,1
1,간호사,3
2,감정∙기술영업및중개관련종사자,21
3,건물 관리원 및 검표원,79
4,건설 및 광업 단순 종사자,96
...,...,...
143,화학∙고무 및 플라스틱 제품 생산기 조작원,7
144,화학공학 기술자 및 시험원,6
145,환경∙청소 및 경비 관련 관리자,10
146,환경공학∙가스·에너지 기술자 및 시험원,4


In [43]:
# 여성 직업 빈도 상위 10개 추출
job_female = welfare.dropna(subset=['job'])\
                    .query('sex=="female"')\
                    .groupby('job', as_index=False)\
                    .agg(n=('job', 'count'))
job_female

,job,n
0,가사 및 육아 도우미,96
1,간호사,70
2,감정∙기술영업및중개관련종사자,6
3,건물 관리원 및 검표원,2
4,건설 및 광업 단순 종사자,8
...,...,...
102,학예사∙사서 및 기록물 관리사,2
103,행정 사무원,54
104,환경∙청소 및 경비 관련 관리자,4
105,환경공학∙가스·에너지 기술자 및 시험원,1


In [48]:
df = job_income.merge(job_male, how='left', on='job')
df = df.merge(job_female, how='left', on='job')
df

,job,mean_income,male_count,female_count,n_x,n_y
0,가사 및 육아 도우미,92.455882,1,96.0,1.0,96.0
1,간호사,265.219178,3,70.0,3.0,70.0
2,감정∙기술영업및중개관련종사자,391.000000,21,6.0,21.0,6.0
3,건물 관리원 및 검표원,168.375000,79,2.0,79.0,2.0
4,건설 및 광업 단순 종사자,261.975000,96,8.0,96.0,8.0
...,...,...,...,...,...,...
142,화학∙고무 및 플라스틱 제품 생산기 조작원,452.714286,1,NaN,7.0,NaN
143,화학공학 기술자 및 시험원,489.500000,7,NaN,6.0,NaN
144,환경∙청소 및 경비 관련 관리자,201.000000,6,NaN,10.0,4.0
145,환경공학∙가스·에너지 기술자 및 시험원,511.000000,10,NaN,4.0,1.0
